# PREPROCESSING

## I.Các bước thực hiện

1. Load 3 file time series COVID-19 (Confirmed, Deaths, Recovered)

2. Làm sạch và chuẩn hóa dữ liệu

3. Gộp theo quốc gia

4. Chuyển dữ liệu từ wide format → long format

5. Merge thành 1 bảng duy nhất

6. Lưu dữ liệu đã xử lý vào thư mục 2_processed

## II.Import thư viện & cấu hình đường dẫn

In [2]:
import pandas as pd
import os

In [37]:
# Đường dẫn thư mục
# Đổi sang đường dẫn tới thư mục data/1_raw từ root của project
RAW_DATA_PATH = "../data/1_raw"
# Lưu kết quả đã xử lý vào data/2_processed
PROCESSED_DATA_PATH = "../data/2_processed"


## III.Step 1 – Load dữ liệu thô

In [6]:
confirmed = pd.read_csv(
    os.path.join(RAW_DATA_PATH, "time_series_covid19_confirmed_global.csv")
)

deaths = pd.read_csv(
    os.path.join(RAW_DATA_PATH, "time_series_covid19_deaths_global.csv")
)

recovered = pd.read_csv(
    os.path.join(RAW_DATA_PATH, "time_series_covid19_recovered_global.csv")
)


### Kiểm tra nhanh dữ liệu thô

In [10]:
print("Confirmed shape:", confirmed.shape)
confirmed.head()

Confirmed shape: (289, 1147)


,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/23,3/1/23,3/2/23,3/3/23,3/4/23,3/5/23,3/6/23,3/7/23,3/8/23,3/9/23
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,209322,209340,209358,209362,209369,209390,209406,209436,209451,209451
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,334391,334408,334408,334427,334427,334427,334427,334427,334443,334457
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,271441,271448,271463,271469,271469,271477,271477,271490,271494,271496
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,47866,47875,47875,47875,47875,47875,47875,47875,47890,47890
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,105255,105277,105277,105277,105277,105277,105277,105277,105288,105288


In [11]:
print("Deaths shape:", deaths.shape)
deaths.head()

Deaths shape: (289, 1147)


,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/23,3/1/23,3/2/23,3/3/23,3/4/23,3/5/23,3/6/23,3/7/23,3/8/23,3/9/23
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,7896,7896,7896,7896,7896,7896,7896,7896,7896,7896
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,3598,3598,3598,3598,3598,3598,3598,3598,3598,3598
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,6881,6881,6881,6881,6881,6881,6881,6881,6881,6881
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,165,165,165,165,165,165,165,165,165,165
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,1933,1933,1933,1933,1933,1933,1933,1933,1933,1933


In [12]:
print("Recovered shape:", recovered.shape)
recovered.head()

Recovered shape: (274, 1147)


,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/23,3/1/23,3/2/23,3/3/23,3/4/23,3/5/23,3/6/23,3/7/23,3/8/23,3/9/23
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## IV.Step 2 – Kiểm tra cấu trúc dữ liệu

In [8]:
confirmed.head()
confirmed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 289 entries, 0 to 288
Columns: 1147 entries, Province/State to 3/9/23
dtypes: float64(2), int64(1143), object(2)
memory usage: 2.5+ MB


Nhận xét cấu trúc

- Dữ liệu ở dạng wide format

- Mỗi cột ngày là số ca cộng dồn

- Nhiều quốc gia có nhiều tỉnh bang → cần gộp

## V.Step 3 – Làm sạch & chuẩn hóa dữ liệu

### Gộp theo quốc gia

In [9]:
confirmed_country = confirmed.groupby(
    "Country/Region"
).sum(numeric_only=True)

deaths_country = deaths.groupby(
    "Country/Region"
).sum(numeric_only=True)

recovered_country = recovered.groupby(
    "Country/Region"
).sum(numeric_only=True)


### Kiểm tra sau khi groupby

In [13]:
print("Confirmed (by country):", confirmed_country.shape)
confirmed_country.head()

Confirmed (by country): (201, 1145)


,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,2/28/23,3/1/23,3/2/23,3/3/23,3/4/23,3/5/23,3/6/23,3/7/23,3/8/23,3/9/23
Country/Region,,,,,,,,,,,,,,,,,,,,,
Afghanistan,33.93911,67.709953,0,0,0,0,0,0,0,0,...,209322,209340,209358,209362,209369,209390,209406,209436,209451,209451
Albania,41.15330,20.168300,0,0,0,0,0,0,0,0,...,334391,334408,334408,334427,334427,334427,334427,334427,334443,334457
Algeria,28.03390,1.659600,0,0,0,0,0,0,0,0,...,271441,271448,271463,271469,271469,271477,271477,271490,271494,271496
Andorra,42.50630,1.521800,0,0,0,0,0,0,0,0,...,47866,47875,47875,47875,47875,47875,47875,47875,47890,47890
Angola,-11.20270,17.873900,0,0,0,0,0,0,0,0,...,105255,105277,105277,105277,105277,105277,105277,105277,105288,105288


In [14]:
print("Deaths (by country):", deaths_country.shape)
deaths_country.head()

Deaths (by country): (201, 1145)


,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,2/28/23,3/1/23,3/2/23,3/3/23,3/4/23,3/5/23,3/6/23,3/7/23,3/8/23,3/9/23
Country/Region,,,,,,,,,,,,,,,,,,,,,
Afghanistan,33.93911,67.709953,0,0,0,0,0,0,0,0,...,7896,7896,7896,7896,7896,7896,7896,7896,7896,7896
Albania,41.15330,20.168300,0,0,0,0,0,0,0,0,...,3598,3598,3598,3598,3598,3598,3598,3598,3598,3598
Algeria,28.03390,1.659600,0,0,0,0,0,0,0,0,...,6881,6881,6881,6881,6881,6881,6881,6881,6881,6881
Andorra,42.50630,1.521800,0,0,0,0,0,0,0,0,...,165,165,165,165,165,165,165,165,165,165
Angola,-11.20270,17.873900,0,0,0,0,0,0,0,0,...,1933,1933,1933,1933,1933,1933,1933,1933,1933,1933


Sau bước này:

- Mỗi dòng = 1 quốc gia

- Khoảng ~180 quốc gia

## Xóa cột Lat / Long

In [15]:
confirmed_country = confirmed_country.drop(
    columns=["Lat", "Long"], errors="ignore"
)

deaths_country = deaths_country.drop(
    columns=["Lat", "Long"], errors="ignore"
)

recovered_country = recovered_country.drop(
    columns=["Lat", "Long"], errors="ignore"
)


### Kiểm tra lại cột

In [17]:
confirmed_country.head()

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,2/28/23,3/1/23,3/2/23,3/3/23,3/4/23,3/5/23,3/6/23,3/7/23,3/8/23,3/9/23
Country/Region,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0,0,0,0,0,0,0,0,0,0,...,209322,209340,209358,209362,209369,209390,209406,209436,209451,209451
Albania,0,0,0,0,0,0,0,0,0,0,...,334391,334408,334408,334427,334427,334427,334427,334427,334443,334457
Algeria,0,0,0,0,0,0,0,0,0,0,...,271441,271448,271463,271469,271469,271477,271477,271490,271494,271496
Andorra,0,0,0,0,0,0,0,0,0,0,...,47866,47875,47875,47875,47875,47875,47875,47875,47890,47890
Angola,0,0,0,0,0,0,0,0,0,0,...,105255,105277,105277,105277,105277,105277,105277,105277,105288,105288


Quan sát

- Chỉ còn Country + các cột ngày

## VI.Step 3C – Wide → Long format

### Confirmed

In [20]:
confirmed_long = confirmed_country.reset_index().melt(
    id_vars="Country/Region",
    var_name="Date",
    value_name="Confirmed"
)
confirmed_long.head()

,Country/Region,Date,Confirmed
0,Afghanistan,1/22/20,0
1,Albania,1/22/20,0
2,Algeria,1/22/20,0
3,Andorra,1/22/20,0
4,Angola,1/22/20,0


### Deaths

In [21]:
deaths_long = deaths_country.reset_index().melt(
    id_vars="Country/Region",
    var_name="Date",
    value_name="Deaths"
)
deaths_long.head()

,Country/Region,Date,Deaths
0,Afghanistan,1/22/20,0
1,Albania,1/22/20,0
2,Algeria,1/22/20,0
3,Andorra,1/22/20,0
4,Angola,1/22/20,0


### Recovered

In [22]:
recovered_long = recovered_country.reset_index().melt(
    id_vars="Country/Region",
    var_name="Date",
    value_name="Recovered"
)
recovered_long.head()

,Country/Region,Date,Recovered
0,Afghanistan,1/22/20,0
1,Albania,1/22/20,0
2,Algeria,1/22/20,0
3,Andorra,1/22/20,0
4,Angola,1/22/20,0


Quan sát

- Mỗi dòng = 1 quốc gia + 1 ngày

- Rất phù hợp cho time series

## VII.Step 3D – Chuyển Date sang datetime

In [24]:
confirmed_long["Date"] = pd.to_datetime(confirmed_long["Date"])
deaths_long["Date"] = pd.to_datetime(deaths_long["Date"])
recovered_long["Date"] = pd.to_datetime(recovered_long["Date"])
confirmed_long.dtypes

Country/Region            object
Date              datetime64[ns]
Confirmed                  int64
dtype: object

Date đã là datetime64

### Save các file data trước khi merge

In [36]:
confirmed_long.to_csv(
    os.path.join(PROCESSED_DATA_PATH, "01_confirmed_long.csv"),
    index=False
)
deaths_long.to_csv(
    os.path.join(PROCESSED_DATA_PATH, "02_deaths_long.csv"),
    index=False
)
recovered_long.to_csv(
    os.path.join(PROCESSED_DATA_PATH, "03_recovered_long.csv"),
    index=False
)
print("✅ Data preprocessing completed!")
print("📁 Output files saved in:", PROCESSED_DATA_PATH)

✅ Data preprocessing completed!
📁 Output files saved in: ../data/2_processed


## VIII.Step 3E – Merge 3 dataset

In [28]:
# Merge deaths
df = confirmed_long.merge(
    deaths_long,
    on=["Country/Region", "Date"],
    how="left"
)
# Merge recovered
df = df.merge(
    recovered_long,
    on=["Country/Region", "Date"],
    how="left"
)
df.head()

,Country/Region,Date,Confirmed,Deaths,Recovered
0,Afghanistan,2020-01-22,0,0,0
1,Albania,2020-01-22,0,0,0
2,Algeria,2020-01-22,0,0,0
3,Andorra,2020-01-22,0,0,0
4,Angola,2020-01-22,0,0,0


## IX.Step 3F – Xử lý missing values

In [29]:
df.isna().sum()
df["Recovered"] = df["Recovered"].fillna(0)
df.isna().sum()

Country/Region    0
Date              0
Confirmed         0
Deaths            0
Recovered         0
dtype: int64

## X.Kiểm tra dữ liệu cuối cùng

In [30]:
df.sample(10)
df.describe()

,Date,Confirmed,Deaths,Recovered
count,229743,2.297430e+05,2.297430e+05,2.297430e+05
mean,2021-08-15 00:00:00,1.379412e+06,1.923809e+04,1.022510e+05
min,2020-01-22 00:00:00,0.000000e+00,0.000000e+00,-1.000000e+00
25%,2020-11-02 00:00:00,3.831000e+03,4.600000e+01,0.000000e+00
50%,2021-08-15 00:00:00,5.293300e+04,7.860000e+02,0.000000e+00
75%,2022-05-28 00:00:00,4.995925e+05,7.227000e+03,3.721500e+03
max,2023-03-09 00:00:00,1.038027e+08,1.123836e+06,3.097475e+07
std,NaN,5.810423e+06,7.592769e+04,8.201866e+05


## XI.Lưu dữ liệu đã clean

In [35]:
output_path = os.path.join(
    PROCESSED_DATA_PATH,
    "04_covid19_cleaned_long.csv"
)

df.to_csv(output_path, index=False)
print("✅ Data preprocessing completed!")
print("📁 Output file:", output_path)

✅ Data preprocessing completed!
📁 Output file: ../data/2_processed\04_covid19_cleaned_long.csv
